In [1]:
import torch 
import os
from torch_geometric.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from tqdm import tqdm
import import_ipynb
from Dataset import MoleculeDataset 
import Model
from Model import GNN
import mlflow.pytorch
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

mlflow.set_tracking_uri("http://localhost:5000")


importing Jupyter notebook from Dataset.ipynb


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
D:\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'haiku'


importing Jupyter notebook from Model.ipynb


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        
        batch.to(device)  
        
        optimizer.zero_grad() 
        
        pred = model(batch.x.float(), 
                                batch.edge_attr.float(),
                                batch.edge_index, 
                                batch.batch) 
        
        loss = loss_fn(torch.squeeze(pred), batch.y.float())
        loss.backward()  
        optimizer.step()  
        
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss/step

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_preds_raw = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)  
        pred = model(batch.x.float(), 
                        batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 
        loss = loss_fn(torch.squeeze(pred), batch.y.float())

        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    print(all_preds_raw[0][:10])
    print(all_preds[:10])
    print(all_labels[:10])
    calculate_metrics(all_preds, all_labels, epoch, "test")
    log_conf_matrix(all_preds, all_labels, epoch)
    return running_loss/step

def log_conf_matrix(y_pred, y_true, epoch):
    
    file_path = f'images/cm_{epoch}.png'
    cm = confusion_matrix(y_pred, y_true)
    classes = ["0", "1"]
    df_cfm = pd.DataFrame(cm, index = classes, columns = classes)
    plt.figure(figsize = (10,7))
    cfm_plot = sns.heatmap(df_cfm, annot=True, cmap='Blues', fmt='g')
    cfm_plot.figure.savefig(file_path)
    if(os.path.exists(file_path)):
        mlflow.log_artifact(f"images/cm_{epoch}.png")
    else:
        print("image not found")
    

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy}")
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    mlflow.log_metric(key=f"Precision-{type}", value=float(prec), step=epoch)
    mlflow.log_metric(key=f"Recall-{type}", value=float(rec), step=epoch)
    mlflow.log_metric(key=f"Accuracy-{type}", value=float(accuracy), step=epoch)
    try:
        roc = roc_auc_score(y_true, y_pred)
        print(f"ROC AUC: {roc}")
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(roc), step=epoch)
    except:
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
        print(f"ROC AUC: notdefined")


from mango import scheduler, tuner
from mango.tuner import Tuner
from config import HYPERPARAMETERS, BEST_PARAMETERS,SIGNATURE

def run_one_training(params):
    params = params[0]
    with mlflow.start_run() as run:
        
        for key in params.keys():
            mlflow.log_param(key, params[key])

        
        train_dataset = MoleculeDataset(root="data/", filename="HIV_train_oversampled.csv")
        test_dataset = MoleculeDataset(root="data/", filename="HIV_test.csv", test=True)
        params["model_edge_dim"] = train_dataset[0].edge_attr.shape[1]

        
        train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=True)

        
        model_params = {k: v for k, v in params.items() if k.startswith("model_")}
        model = GNN(feature_size=train_dataset[0].x.shape[1], model_params=model_params) 
        model = model.to(device)
        print(f"Number of parameters: {count_parameters(model)}")
        mlflow.log_param("num_params", count_parameters(model))

    
        weight = torch.tensor([params["pos_weight"]], dtype=torch.float32).to(device)
        loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=weight)
        optimizer = torch.optim.SGD(model.parameters(), 
                                    lr=params["learning_rate"],
                                    momentum=params["sgd_momentum"],
                                    weight_decay=params["weight_decay"])
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params["scheduler_gamma"])
        
        
        best_loss = 1000
        early_stopping_counter = 0
        for epoch in range(300): 
            if early_stopping_counter <= 10: 
                
                model.train()
                loss = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
                print(f"Epoch {epoch} | Train Loss {loss}")
                mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

                
                model.eval()
                if epoch % 5 == 0:
                    loss = test(epoch, model, test_loader, loss_fn)
                    print(f"Epoch {epoch} | Test Loss {loss}")
                    mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
                    
                    
                    if float(loss) < best_loss:
                        best_loss = loss
                        
                        mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
                        early_stopping_counter = 0
                    else:
                        early_stopping_counter += 1

                scheduler.step()
            else:
                
                return [best_loss]
    print(f"Finishing training with best test loss: {best_loss}")
    return [best_loss]



config = dict()
config["optimizer"] = "Bayesian"
config["num_iteration"] = 100

tuner = Tuner(BEST_PARAMETERS, 
              objective=run_one_training,
              conf_dict=config) 
results = tuner.minimize()

Number of parameters: 365633


100%|██████████| 560/560 [07:36<00:00,  1.23it/s]



 Confusion matrix: 
 [[13649  9068]
 [22201 26716]]
F1 Score: 0.6308308048311118
Accuracy: 0.5634894044727364
Precision: 0.5461496003434388
Recall: 0.7465906550413592
ROC AUC: 0.5636579495569418
Epoch 0 | Train Loss 0.7564117217702525
[[0.4885675 ]
 [0.5022531 ]
 [0.42802837]
 [0.6394342 ]
 [0.42966688]
 [0.50174105]
 [0.6236231 ]
 [0.56727153]
 [0.46092224]
 [0.4380293 ]]
[0. 1. 0. 1. 0. 1. 1. 1. 0. 0.]
[0 0 0 0 0 0 0 0 0 0]

 Confusion matrix: 
 [[2563   32]
 [1271  133]]
F1 Score: 0.1695347355003187
Accuracy: 0.6741685421355339
Precision: 0.09472934472934473
Recall: 0.806060606060606
ROC AUC: 0.737276521079338
Epoch 0 | Test Loss 0.6828329432755709


2024/05/04 23:02:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\User\AppData\Local\Temp\tmp1cr1ujcy\model\data, flavor: pytorch). Fall back to return ['torch==2.2.1', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
100%|██████████| 560/560 [05:02<00:00,  1.85it/s]



 Confusion matrix: 
 [[23843 12679]
 [12007 23105]]
F1 Score: 0.6517998194538479
Accuracy: 0.6553871066811849
Precision: 0.6580371383002962
Recall: 0.6456796333556897
ROC AUC: 0.6553781709316803
Epoch 1 | Train Loss 0.7056518653673786


...

100%|██████████| 560/560 [01:38<00:00,  5.67it/s]



 Confusion matrix: 
 [[31224 12054]
 [ 4626 23730]]
F1 Score: 0.7399438727782974
Accuracy: 0.7671496775274311
Precision: 0.8368599238256453
Recall: 0.6631455399061033
ROC AUC: 0.7670539415011688
Epoch 119 | Train Loss 0.5760394100099802


100%|██████████| 560/560 [01:38<00:00,  5.67it/s]



 Confusion matrix: 
 [[31111 12053]
 [ 4739 23731]]
F1 Score: 0.7386621844554423
Accuracy: 0.7655861741631069
Precision: 0.8335440814892869
Recall: 0.6631734853565839
ROC AUC: 0.765491903068808
Epoch 120 | Train Loss 0.5799134453492505
[[0.31881884]
 [0.39629203]
 [0.85688144]
 [0.3355302 ]
 [0.40845862]
 [0.6289921 ]
 [0.3199279 ]
 [0.3153459 ]
 [0.3120693 ]
 [0.32206655]]
[0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0]

 Confusion matrix: 
 [[3555   69]
 [ 279   96]]
F1 Score: 0.35555555555555557
Accuracy: 0.9129782445611403
Precision: 0.256
Recall: 0.5818181818181818
ROC AUC: 0.7545241143832693
Epoch 120 | Test Loss 0.48019166104495525


  9%|▉         | 51/560 [00:08<01:41,  5.02it/s]